In [10]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from matplotlib.colors import LightSource, LinearSegmentedColormap
import matplotlib.cm as cm
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pyvista.jupyter")
warnings.filterwarnings("ignore", message=".*tight_layout.*")

# Define a more natural terrain colormap with appropriate elevation boundaries
def create_terrain_cmap():
    # Colors for different terrain types from low to high elevation
    colors = [
        (0.0, (0.0, 0.0, 0.5)),      # Deep blue (ocean)
        (0.1, (0.0, 0.4, 0.8)),      # Medium blue (deeper water)
        (0.2, (0.2, 0.6, 0.9)),      # Light blue (shallow water)
        (0.3, (0.8, 0.8, 0.2)),      # Sand/beach
        (0.4, (0.1, 0.6, 0.1)),      # Lowland green
        (0.6, (0.4, 0.8, 0.4)),      # Highland green
        (0.7, (0.6, 0.5, 0.3)),      # Low mountains (brown)
        (0.8, (0.5, 0.4, 0.3)),      # High mountains (darker brown)
        (0.9, (0.9, 0.9, 0.9)),      # Lower snow regions
        (1.0, (1.0, 1.0, 1.0))       # Upper snow regions (pure white)
    ]
    
    cdict = {'red': [], 'green': [], 'blue': []}
    for pos, color in colors:
        cdict['red'].append((pos, color[0], color[0]))
        cdict['green'].append((pos, color[1], color[1]))
        cdict['blue'].append((pos, color[2], color[2]))
    
    return LinearSegmentedColormap('terrain_natural', cdict)

# Create the natural terrain colormap
terrain_cmap = create_terrain_cmap()

try:
    import pyvista as pv
    HAS_PYVISTA = True
except ImportError:
    HAS_PYVISTA = False
    print("PyVista not installed—install with `pip install pyvista` for interactive viewing")

try:
    from opensimplex import OpenSimplex
    HAS_OPENSIMPLEX = True
except ImportError:
    HAS_OPENSIMPLEX = False
    print("OpenSimplex not installed—install with `pip install opensimplex` for better fractal detail")

# Create output directory
OUTPUT_DIR = "Generated_3D"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_height_map(path, scale_to=None, invert=True):
    img = Image.open(path).convert("L")
    arr = np.array(img, dtype=np.float32) / 255.0
    if invert:
        arr = 1.0 - arr
    if scale_to is not None:
        hi, lo = scale_to
        arr = arr * (hi - lo) + lo
    return arr

def smooth_terrain(height_map, sigma=1.0):
    return gaussian_filter(height_map, sigma=sigma)

def add_opensimplex_detail(height_map, octaves=3, persistence=0.5, scale=10.0):
    if not HAS_OPENSIMPLEX:
        return add_numpy_fractal_detail(height_map, octaves, persistence, scale)
    h, w = height_map.shape
    detail = np.zeros((h, w))
    noise_gen = OpenSimplex(seed=np.random.randint(0, 1000000))
    for y in range(h):
        for x in range(w):
            value = 0
            amplitude = 1.0
            frequency = 1.0
            for _ in range(octaves):
                nx = x / w * frequency * 5
                ny = y / h * frequency * 5
                value += noise_gen.noise2(nx, ny) * amplitude
                amplitude *= persistence
                frequency *= 2
            detail[y, x] = value
    detail = (detail - detail.min()) / (detail.max() - detail.min()) * scale
    return height_map + detail

def add_numpy_fractal_detail(height_map, octaves=3, persistence=0.5, scale=10.0):
    h, w = height_map.shape
    detail = np.zeros((h, w))
    for octave in range(octaves):
        freq = 2**octave
        amp = persistence**octave
        octave_size = (max(h // freq, 1), max(w // freq, 1))
        noise = np.random.rand(*octave_size)
        from scipy.ndimage import zoom
        zoomed = zoom(noise, (h/noise.shape[0], w/noise.shape[1]), order=1)
        detail += zoomed * amp
    detail = (detail - detail.min()) / (detail.max() - detail.min()) * scale
    return height_map + detail

def enhance_terrain(height_map, smooth_sigma=1.0, add_detail=True, 
                   detail_octaves=3, detail_persistence=0.5, detail_scale=10.0):
    terrain = smooth_terrain(height_map, sigma=smooth_sigma)
    if add_detail:
        if HAS_OPENSIMPLEX:
            terrain = add_opensimplex_detail(terrain, octaves=detail_octaves,
                                           persistence=detail_persistence, scale=detail_scale)
        else:
            terrain = add_numpy_fractal_detail(terrain, octaves=detail_octaves,
                                             persistence=detail_persistence, scale=detail_scale)
    terrain = np.clip(terrain, 0, 200)
    if terrain.max() > terrain.min():
        terrain = (terrain - terrain.min()) / (terrain.max() - terrain.min()) * 200
    else:
        terrain = np.zeros_like(terrain)
    return terrain

def plot_matplotlib_3d(z, title="3D Terrain", cmap=terrain_cmap, elev=30, azim=-60, with_hillshade=True, output_prefix=""):
    """Plot 3D terrain with improved coloring and optional hillshading"""
    # Create path within output directory
    filename = os.path.join(OUTPUT_DIR, f"{output_prefix}_3D_Terrain.png")
    
    cmap_obj = cmap if isinstance(cmap, LinearSegmentedColormap) else cm.get_cmap(cmap)
    h, w = z.shape
    x = np.arange(w)
    y = np.arange(h)
    x, y = np.meshgrid(x, y)
    
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection="3d")
    
    # Normalize z values for coloring
    norm_z = (z - z.min()) / (z.max() - z.min() if z.max() > z.min() else 1)
    
    if with_hillshade:
        # Create hillshading for better depth perception
        ls = LightSource(azdeg=315, altdeg=45)
        rgb = ls.shade(z, cmap=cmap_obj, vert_exag=0.3, blend_mode='soft')
        surf = ax.plot_surface(x, y, z, facecolors=rgb, linewidth=0, antialiased=True,
                              rcount=200, ccount=200)
    else:
        surf = ax.plot_surface(x, y, z, cmap=cmap_obj, linewidth=0, antialiased=True,
                              rcount=200, ccount=200, vmin=0, vmax=200)
    
    # Create a colorbar by directly passing the surface object
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="Elevation")
    
    ax.set_title(title, fontsize=16)
    ax.set_xlabel("X", fontsize=12)
    ax.set_ylabel("Y", fontsize=12)
    ax.set_zlabel("Elevation", fontsize=12)
    ax.set_zlim(0, 200)
    ax.view_init(elev=elev, azim=azim)
    
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Saved: {filename}")
    plt.close()

def plot_comparison(original, enhanced, titles=["Original", "Enhanced"], elev=30, azim=-60, cmap=terrain_cmap, output_prefix=""):
    """Plot comparison of original and enhanced terrain with improved coloring"""
    # Create path within output directory
    filename = os.path.join(OUTPUT_DIR, f"{output_prefix}_terrain_comparison.png")
    
    cmap_obj = cmap if isinstance(cmap, LinearSegmentedColormap) else cm.get_cmap(cmap)
    h, w = original.shape
    x = np.arange(w)
    y = np.arange(h)
    x, y = np.meshgrid(x, y)
    
    fig = plt.figure(figsize=(18, 8))
    
    # First terrain plot (original)
    ax1 = fig.add_subplot(121, projection="3d")
    ls = LightSource(azdeg=315, altdeg=45)
    rgb1 = ls.shade(original, cmap=cmap_obj, vert_exag=0.3, blend_mode='soft')
    surf1 = ax1.plot_surface(x, y, original, facecolors=rgb1, linewidth=0, antialiased=True,
                            rcount=200, ccount=200)
    ax1.set_title(titles[0], fontsize=16)
    ax1.set_zlim(0, 200)
    ax1.view_init(elev=elev, azim=azim)
    ax1.set_xlabel("X", fontsize=10)
    ax1.set_ylabel("Y", fontsize=10)
    ax1.set_zlabel("Elevation", fontsize=10)
    
    # Second terrain plot (enhanced)
    ax2 = fig.add_subplot(122, projection="3d")
    rgb2 = ls.shade(enhanced, cmap=cmap_obj, vert_exag=0.3, blend_mode='soft')
    surf2 = ax2.plot_surface(x, y, enhanced, facecolors=rgb2, linewidth=0, antialiased=True,
                            rcount=200, ccount=200)
    ax2.set_title(titles[1], fontsize=16)
    ax2.set_zlim(0, 200)
    ax2.view_init(elev=elev, azim=azim)
    ax2.set_xlabel("X", fontsize=10)
    ax2.set_ylabel("Y", fontsize=10)
    ax2.set_zlabel("Elevation", fontsize=10)
    
    # Create a colorbar using the surface object directly
    fig.colorbar(surf2, ax=[ax1, ax2], shrink=0.5, aspect=10, label="Elevation")
    
    plt.subplots_adjust(wspace=0.35, left=0.05, right=0.95)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Saved: {filename}")
    plt.close()

def plot_all_variations(height_map, smooth_sigmas=[0.0, 1.0, 3.0], titles=None, 
                       elev=30, azim=-60, cmap=terrain_cmap, output_prefix=""):
    """Plot different smoothing variations with improved coloring"""
    # Create path within output directory
    filename = os.path.join(OUTPUT_DIR, f"{output_prefix}_terrain_variations.png")
    
    cmap_obj = cmap if isinstance(cmap, LinearSegmentedColormap) else cm.get_cmap(cmap)
    n = len(smooth_sigmas)
    fig = plt.figure(figsize=(18, 6))
    
    if titles is None:
        titles = [f"Smoothing: {s}" for s in smooth_sigmas]
        
    h, w = height_map.shape
    x = np.arange(w)
    y = np.arange(h)
    x, y = np.meshgrid(x, y)
    
    ls = LightSource(azdeg=315, altdeg=45)
    
    axes = []
    surfaces = []
    
    for i, sigma in enumerate(smooth_sigmas):
        processed = smooth_terrain(height_map, sigma=sigma)
        ax = fig.add_subplot(1, n, i+1, projection="3d")
        axes.append(ax)
        
        # Apply hillshading
        rgb = ls.shade(processed, cmap=cmap_obj, vert_exag=0.3, blend_mode='soft')
        surf = ax.plot_surface(x, y, processed, facecolors=rgb, linewidth=0, antialiased=True,
                              rcount=150, ccount=150)
        surfaces.append(surf)
        
        ax.set_title(titles[i], fontsize=14)
        ax.set_zlim(0, 200)
        ax.view_init(elev=elev, azim=azim)
        ax.set_xlabel("X", fontsize=8)
        ax.set_ylabel("Y", fontsize=8)
        ax.set_zlabel("Elevation", fontsize=8)
    
    # Adding the colorbar with the last surface
    if surfaces:
        fig.colorbar(surfaces[-1], ax=axes, shrink=0.5, aspect=10, label="Elevation")
    
    plt.subplots_adjust(wspace=0.4, left=0.05, right=0.95)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Saved: {filename}")
    plt.close()

def plot_pyvista_terrain(z, name="terrain", output_prefix=""):
    """Plot terrain using PyVista with improved coloring"""
    if not HAS_PYVISTA:
        print("PyVista not available. Skipping interactive plot.")
        return
        
    h, w = z.shape
    x = np.arange(w)
    y = np.arange(h)
    x, y = np.meshgrid(x, y)
    pts = np.column_stack((x.flatten(), y.flatten(), z.flatten()))
    
    # Create structured grid
    grid = pv.StructuredGrid()
    grid.points = pts
    grid.dimensions = (w, h, 1)
    
    # Add scalars to the grid (needed for coloring)
    grid.point_data["elevation"] = z.flatten()
    
    # Create plotter with improved settings
    pl = pv.Plotter(window_size=[1024, 768])
    
    # Use terrain-like colormap
    # Convert matplotlib colormap to PyVista format if needed
    try:
        pv_cmap = terrain_cmap
    except (AttributeError, TypeError):
        # Default to PyVista's terrain if conversion fails
        pv_cmap = 'terrain'
    
    # Apply better coloring
    pl.add_mesh(grid, scalars="elevation", show_edges=False, cmap=pv_cmap, 
                smooth_shading=True, scalar_bar_args={'title': 'Elevation'})
    
    # Add lights if possible
    try:
        pl.add_light(pv.Light(position=(1, 1, 1), intensity=0.7))
        pl.add_light(pv.Light(position=(-1, -1, 1), intensity=0.3))
    except AttributeError:
        # Older versions of PyVista
        pass
    
    # Try to enable enhanced visualization if available
    try:
        pl.enable_ambient_occlusion(ambient=0.3)
    except (AttributeError, ImportError):
        pass
    
    pl.show_grid()
    pl.view_isometric()
    pl.show(title=name)

def plot_2d_terrain(z, title="2D Terrain Map", cmap=terrain_cmap, output_prefix=""):
    """Create a 2D shaded relief map of the terrain"""
    # Create path within output directory
    filename = os.path.join(OUTPUT_DIR, f"{output_prefix}_2D_Relief_Map.png")
    
    ls = LightSource(azdeg=315, altdeg=45)
    rgb = ls.shade(z, cmap=cmap, vert_exag=0.3, blend_mode='soft')
    
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb)
    plt.axis('off')
    plt.title(title, fontsize=16)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Saved: {filename}")
    plt.close()

def process_terrain_from_scratch(size=(256, 256), scale=200, detail_octaves=6, 
                                detail_persistence=0.5, detail_scale=10.0):
    """Generate terrain from scratch using fractal noise"""
    # Create base noise
    base = np.zeros(size)
    # Add fractal detail
    if HAS_OPENSIMPLEX:
        terrain = add_opensimplex_detail(base, octaves=detail_octaves,
                                      persistence=detail_persistence, scale=detail_scale)
    else:
        terrain = add_numpy_fractal_detail(base, octaves=detail_octaves,
                                        persistence=detail_persistence, scale=detail_scale)
    # Scale to desired height range
    terrain = (terrain - terrain.min()) / (terrain.max() - terrain.min()) * scale
    return terrain

def process_all_terrain_examples(directory, max_examples=5, invert=True, smooth_sigma=3.0,
                               detail_octaves=4, detail_persistence=0.6, detail_scale=10.0,
                               show_comparison=True, show_pyvista=False, elev=30, azim=-60):
    """Process all terrain examples with improved coloring"""
    # First check if directory exists, if not create a sample terrain
    if not os.path.exists(directory):
        print(f"Directory not found: {directory}")
        print("Generating sample terrain instead...")
        sample_terrain = process_terrain_from_scratch(size=(256, 256), 
                                                    detail_octaves=5, 
                                                    detail_persistence=0.5,
                                                    detail_scale=15.0)
        
        # Save and visualize the sample terrain
        plot_matplotlib_3d(sample_terrain, title="Generated Sample Terrain",
                           cmap=terrain_cmap, elev=elev, azim=azim, with_hillshade=True,
                           output_prefix="sample")
        
        plot_2d_terrain(sample_terrain, title="2D Sample Terrain Map",
                       output_prefix="sample")
        
        if show_pyvista and HAS_PYVISTA:
            print("Launching PyVista 3D viewer for generated terrain...")
            plot_pyvista_terrain(sample_terrain, name="Generated Terrain",
                               output_prefix="sample")
        
        return
        
    valid_extensions = ['.png', '.jpg', '.jpeg', '.tif', '.tiff']
    files = [f for f in os.listdir(directory) if os.path.splitext(f.lower())[1] in valid_extensions]
    
    if not files:
        print(f"No valid image files found in {directory}")
        return
        
    files = sorted(files)[:max_examples]
    print(f"Found {len(files)} image files to process.")
    
    for fn in files:
        path = os.path.join(directory, fn)
        print(f"\n--- Processing {fn} ---")
        
        # Extract filename without extension to use as prefix for output files
        file_prefix = os.path.splitext(fn)[0]
        
        try:
            z_original = load_height_map(path, scale_to=(200, 0), invert=invert)
            z_enhanced = enhance_terrain(z_original, smooth_sigma=smooth_sigma, add_detail=True,
                                        detail_octaves=detail_octaves, detail_persistence=detail_persistence,
                                        detail_scale=detail_scale)
            
            # Generate 2D shaded relief map for quick preview with unique filename
            plot_2d_terrain(z_enhanced, title=f"2D Relief Map", output_prefix=file_prefix)
            
            if show_comparison:
                plot_comparison(z_original, z_enhanced, 
                               titles=[f"Original", "Enhanced with Smoothing & Fractal Detail"],
                               elev=elev, azim=azim, cmap=terrain_cmap, output_prefix=file_prefix)
            
            # Create a high-quality standalone 3D rendering with unique filename
            plot_matplotlib_3d(z_enhanced, title=f"Enhanced 3D Terrain", 
                              cmap=terrain_cmap, elev=elev, azim=azim, with_hillshade=True, 
                              output_prefix=file_prefix)
            
            if show_pyvista and HAS_PYVISTA:
                print("Launching PyVista 3D viewer for enhanced terrain...")
                plot_pyvista_terrain(z_enhanced, name=f"Enhanced {fn}", output_prefix=file_prefix)
                
            plot_all_variations(z_original, smooth_sigmas=[0.0, 1.0, 3.0, 5.0],
                               titles=["No Smoothing", "Light Smoothing", "Medium Smoothing", "Heavy Smoothing"],
                               elev=elev, azim=azim, cmap=terrain_cmap, output_prefix=file_prefix)
            
            print(f"Successfully processed {fn}")
        
        except Exception as e:
            print(f"Error processing {fn}: {e}")
            continue

def main():
    gen_dir = "gen_images"
    
    # Create the directory if it doesn't exist
    if not os.path.exists(gen_dir):
        print(f"Creating directory: {gen_dir}")
        os.makedirs(gen_dir)
    
    # Check if directory is empty, generate a sample terrain if needed
    if not os.listdir(gen_dir):
        print(f"No images found in {gen_dir}. Generating a sample terrain...")
        sample_terrain = process_terrain_from_scratch(size=(256, 256))
        # Save the sample terrain as an image
        sample_img = (sample_terrain / sample_terrain.max() * 255).astype(np.uint8)
        img = Image.fromarray(sample_img)
        img.save(os.path.join(gen_dir, "sample_terrain.png"))
        print(f"Saved sample terrain to {gen_dir}/sample_terrain.png")
    
    print(f"Output images will be saved to: {os.path.abspath(OUTPUT_DIR)}")
    
    process_all_terrain_examples(directory=gen_dir, max_examples=5, invert=True, smooth_sigma=3.0,
                                detail_octaves=4, detail_persistence=0.5, detail_scale=10.0,
                                show_comparison=True, show_pyvista=False, elev=30, azim=-60)
    
    print(f"\nProcessing complete! All images have been saved to the {OUTPUT_DIR} folder.")

if __name__ == "__main__":
    main()

Output images will be saved to: d:\GAN\GAN\Generated_3D
Found 5 image files to process.

--- Processing 0000.png ---
Saved: Generated_3D\0000_2D_Relief_Map.png
Saved: Generated_3D\0000_terrain_comparison.png
Saved: Generated_3D\0000_3D_Terrain.png
Saved: Generated_3D\0000_terrain_variations.png
Successfully processed 0000.png

--- Processing 0001.png ---
Saved: Generated_3D\0001_2D_Relief_Map.png
Saved: Generated_3D\0001_terrain_comparison.png
Saved: Generated_3D\0001_3D_Terrain.png
Saved: Generated_3D\0001_terrain_variations.png
Successfully processed 0001.png

--- Processing 0002.png ---
Saved: Generated_3D\0002_2D_Relief_Map.png
Saved: Generated_3D\0002_terrain_comparison.png
Saved: Generated_3D\0002_3D_Terrain.png
Saved: Generated_3D\0002_terrain_variations.png
Successfully processed 0002.png

--- Processing 0003.png ---
Saved: Generated_3D\0003_2D_Relief_Map.png
Saved: Generated_3D\0003_terrain_comparison.png
Saved: Generated_3D\0003_3D_Terrain.png
Saved: Generated_3D\0003_terrain